In [8]:
pip install timesfm

In [9]:
# Import necessary libraries
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt # Plotting removed for iterative runs
from sklearn.metrics import mean_absolute_error, mean_squared_error
import timesfm  # Make sure timesfm is installed (pip install timesfm)
import time # To time each iteration

In [10]:
# --- Configuration Variables ---
# Define the LISTS of historical data lengths (context) to test
INPUT_CONTEXT_LENGTHS = [2048, 2368, 2688, 3200, 3712, 4096]

# Define the LISTS of future time steps (horizon) to predict
FORECAST_HORIZON_LENGTHS = [64, 128, 192, 256, 480, 512]

# Define the Hugging Face repository ID for the desired TimesFM checkpoint
CHECKPOINT_REPO_ID = "google/timesfm-2.0-500m-pytorch"

# Path to the dataset CSV file
DATASET_PATH = "/content/weatherHistory.csv" # Make sure this path is correct

# Column name containing the time series data
DATA_COLUMN_NAME = 'Temperature (C)'

# Backend for TimesFM ('gpu' or 'cpu')
TIMESFM_BACKEND = "gpu"

In [11]:
# --- Code Execution ---

print(f"Starting iterative forecasting runs...")
print(f"Context Lengths to test: {INPUT_CONTEXT_LENGTHS}")
print(f"Horizon Lengths to test: {FORECAST_HORIZON_LENGTHS}")
print("-" * 30)

# 1. Load the dataset (only once)
print(f"Loading data from: {DATASET_PATH}")
data = pd.read_csv(DATASET_PATH)

# Convert the target data to a NumPy array (only once)
temperature_data = data[DATA_COLUMN_NAME].values  # Shape: (num_samples,)
total_data_length = len(temperature_data)
print(f"Extracted '{DATA_COLUMN_NAME}' data. Total length: {total_data_length}")

# Get units from column name (assuming format 'Name (Unit)')
try:
    data_units = data[DATA_COLUMN_NAME].name.split('(')[-1][:-1]
except:
    data_units = "units" # Default if parsing fails

Starting iterative forecasting runs...
Context Lengths to test: [2048, 2368, 2688, 3200, 3712, 4096]
Horizon Lengths to test: [64, 128, 192, 256, 480, 512]
------------------------------
Loading data from: /content/weatherHistory.csv
Extracted 'Temperature (C)' data. Total length: 96453


In [12]:
# 2. Initialize results storage
results = {
    'Context Length': [],
    'Horizon Length': [],
    'MAE': [],
    'RMSE': [],
    'Time (s)': []
}

# 3. Iterate through each combination of context and horizon lengths
for context_len in INPUT_CONTEXT_LENGTHS:
    for horizon_len in FORECAST_HORIZON_LENGTHS:
        start_time = time.time()
        print(f"\n--- Running for Context={context_len}, Horizon={horizon_len} ---")

        required_length = context_len + horizon_len

        # 3.1 Initialize the TimeSFM model (needs re-initialization for different params)
        print(f"Initializing TimesFm model...")
        try:
            tm = timesfm.TimesFm(
                hparams=timesfm.TimesFmHparams(
                    backend=TIMESFM_BACKEND,
                    per_core_batch_size=64, # Adjust batch size based on GPU memory if needed
                    horizon_len=horizon_len,
                    num_layers=50, # Keep other params consistent or make them configurable too
                    use_positional_embedding=False,
                    context_len=context_len,
                ),
                checkpoint=timesfm.TimesFmCheckpoint(
                    huggingface_repo_id=CHECKPOINT_REPO_ID
                ),
            )
            print("Model initialized.")
        except Exception as e:
            print(f"Error initializing model for Context={context_len}, Horizon={horizon_len}: {e}")
            continue # Skip to next iteration if model init fails

        # 3.2 Prepare Input and Actual Data Slices for this iteration
        input_context = temperature_data[:context_len]
        actual_temperatures = temperature_data[context_len : context_len + horizon_len]
        print(f"Input context shape: {input_context.shape}")
        print(f"Actual temperatures shape: {actual_temperatures.shape}")

        forecast_input = [input_context]
        frequency_input = [1] # hourly frequency

        # 3.3 Generate Predictions
        print("Generating forecast...")
        try:
            point_forecast, _ = tm.forecast(
                forecast_input,
                freq=frequency_input,
            )
            predicted_temperatures = point_forecast[0]
            print(f"Forecast generated. Prediction shape: {predicted_temperatures.shape}")

            # Sanity check for prediction length
            if len(predicted_temperatures) != horizon_len:
                 print(f"Warning: Predicted length ({len(predicted_temperatures)}) does not match "
                       f"the expected horizon length ({horizon_len}). Evaluation might be inaccurate.")
                 if len(actual_temperatures) > len(predicted_temperatures):
                     actual_temperatures = actual_temperatures[:len(predicted_temperatures)]
                     print(f"Trimmed actual_temperatures to length {len(actual_temperatures)} to match prediction.")


            # 3.4 Evaluate the Forecast
            print("Evaluating forecast...")
            mae = mean_absolute_error(actual_temperatures, predicted_temperatures)
            rmse = np.sqrt(mean_squared_error(actual_temperatures, predicted_temperatures))
            print(f"MAE: {mae:.3f} {data_units}")
            print(f"RMSE: {rmse:.3f} {data_units}")

            # 3.5 Store Results
            end_time = time.time()
            iteration_time = end_time - start_time
            results['Context Length'].append(context_len)
            results['Horizon Length'].append(horizon_len)
            results['MAE'].append(mae)
            results['RMSE'].append(rmse)
            results['Time (s)'].append(iteration_time)
            print(f"Iteration completed in {iteration_time:.2f} seconds.")

        except Exception as e:
            print(f"Error during forecast/evaluation for Context={context_len}, Horizon={horizon_len}: {e}")
            # Optionally store NaN or skip storing results for this failed run
            end_time = time.time()
            iteration_time = end_time - start_time
            results['Context Length'].append(context_len)
            results['Horizon Length'].append(horizon_len)
            results['MAE'].append(np.nan) # Indicate failure with NaN
            results['RMSE'].append(np.nan)
            results['Time (s)'].append(iteration_time)


--- Running for Context=2048, Horizon=64 ---
Initializing TimesFm model...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Model initialized.
Input context shape: (2048,)
Actual temperatures shape: (64,)
Generating forecast...
Forecast generated. Prediction shape: (64,)
Evaluating forecast...
MAE: 8.720 C
RMSE: 9.529 C
Iteration completed in 10.23 seconds.

--- Running for Context=2048, Horizon=128 ---
Initializing TimesFm model...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Model initialized.
Input context shape: (2048,)
Actual temperatures shape: (128,)
Generating forecast...
Forecast generated. Prediction shape: (128,)
Evaluating forecast...
MAE: 9.648 C
RMSE: 10.185 C
Iteration completed in 6.96 seconds.

--- Running for Context=2048, Horizon=192 ---
Initializing TimesFm model...


...

Model initialized.
Input context shape: (4096,)
Actual temperatures shape: (256,)
Generating forecast...
Forecast generated. Prediction shape: (256,)
Evaluating forecast...
MAE: 8.240 C
RMSE: 10.232 C
Iteration completed in 10.23 seconds.

--- Running for Context=4096, Horizon=480 ---
Initializing TimesFm model...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Model initialized.
Input context shape: (4096,)
Actual temperatures shape: (480,)
Generating forecast...
Forecast generated. Prediction shape: (480,)
Evaluating forecast...
MAE: 8.689 C
RMSE: 11.009 C
Iteration completed in 14.93 seconds.

--- Running for Context=4096, Horizon=512 ---
Initializing TimesFm model...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Model initialized.
Input context shape: (4096,)
Actual temperatures shape: (512,)
Generating forecast...
Forecast generated. Prediction shape: (512,)
Evaluating forecast...
MAE: 9.073 C
RMSE: 11.385 C
Iteration completed in 15.01 seconds.


In [15]:
# 4. Create and Print DataFrame
print("\n--- All Iterations Complete ---")
results_df = pd.DataFrame(results)

# Format the floating point numbers for better readability
results_df['MAE'] = results_df['MAE'].map('{:.3f}'.format)
results_df['RMSE'] = results_df['RMSE'].map('{:.3f}'.format)
results_df['Time (s)'] = results_df['Time (s)'].map('{:.2f}'.format)

# print("\nForecasting Results Summary:")
# print(results_df.to_string(index=False))
results_df


--- All Iterations Complete ---


,Context Length,Horizon Length,MAE,RMSE,Time (s)
0,2048,64,8.720,9.529,10.23
1,2048,128,9.648,10.185,6.96
2,2048,192,8.731,9.826,7.88
3,2048,256,6.948,8.557,7.66
4,2048,480,6.103,7.727,11.21
5,2048,512,6.311,7.851,11.17
6,2368,64,4.779,5.043,6.68
7,2368,128,5.536,6.148,6.54
8,2368,192,6.165,6.696,9.50
9,2368,256,5.646,6.177,9.47
